<a href="https://colab.research.google.com/github/mgamzec/GeoPython/blob/main/geemap_unsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=-x3wCUvk_Q2HjagiZ4-gQWpSsvYQMHmQq6Ectivehuo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgbsZZbo1kPBE4jF0vjbD_TIpyEe_yy_CFDWVe31zPhesT_HA5_5DU

Successfully saved authorization token.


In [ ]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [ ]:
wdpa_polygon = ee.FeatureCollection('WCMC/WDPA/current/polygons')
# select the pa in cambodia
aoi = wdpa_polygon.filter(ee.Filter.eq('NAME', 'Ang Trapeng Thmor'))


In [ ]:
#image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    #.filterBounds(aoi) \
    #.filterDate('2019-01-01', '2019-12-31') \
   # .sort('CLOUD_COVER') \
    #.first() \
    #.select('B[1-7]')

#image=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(aoi).filterDate('2020-01-01', '2020-12-31').sort('CLOUD_COVER').first().select('B[1-7]')
image=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(aoi).filterDate('2020-01-01', '2020-12-31').sort('CLOUD_COVER').first().select('B[1-7]')


vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

map4 = geemap.Map(center=[12.5657, 105],zoom=5)
map4.add_basemap('SATELLITE')
map4.addLayer(image, vis_params, 'Landsat-8')

map4.addLayerControl()
map4

Map(center=[12.5657, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

In [ ]:
props = geemap.image_props(image)
props.getInfo()

{'CLOUD_COVER': 0.01,
 'CLOUD_COVER_LAND': 0.01,
 'EARTH_SUN_DISTANCE': 0.988817,
 'ESPA_VERSION': '2_23_0_1b',
 'GEOMETRIC_RMSE_MODEL': 7.322,
 'GEOMETRIC_RMSE_MODEL_X': 5.001,
 'GEOMETRIC_RMSE_MODEL_Y': 5.348,
 'IMAGE_DATE': '2020-02-21',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_127050_20200221_20200225_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1582650137000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2020-02-21T03:25:25.1175300Z',
 'SOLAR_AZIMUTH_ANGLE': 130.555389,
 'SOLAR_ZENITH_ANGLE': 37.660316,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 127,
 'WRS_ROW': 50,
 'system:asset_size': '645.595603 MB',
 'system:band_names': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_127050_20200221',
 'system:index': 'LC08_127050_20200221',
 'system:time_end': '2020-02-21 03:25:25',
 'system:time_start': '2020-02-21 03:25:25',
 'system:version': 158304890

In [ ]:
# Make the training dataset.
#training = image.sample(**{
   # 'region': aoi,
   # 'scale': 30,
   # 'numPixels': 5000,
  #  'seed': 0,
   # 'geometries': True  # Set this to False to ignore geometries
#})

training = image.sample(**{
    'region':aoi,
    'scale':30,
    'numPixels':5000,
    'seed':0,
    'geometries': True
})

map4.addLayer(training, {}, 'training', False)
map4

Map(bottom=30462.0, center=[14.168534269226576, 103.831787109375], controls=(WidgetControl(options=['position'…

Train the clusterer

In [ ]:
# Instantiate the clusterer and train it.
#n_clusters = 5
#clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)
n_clusters = 6
clusterer=ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [ ]:
# Cluster the input using the trained clusterer.
#result = image.cluster(clusterer)
result=image.cluster(clusterer)

# # Display the clusters with random colors.
#map4.addLayer(result.randomVisualizer(), {}, 'clusters')
map4.addLayer(result.randomVisualizer(),{},'clusters')
map4

Map(bottom=15487.0, center=[13.036669323115246, 102.59033203125001], controls=(WidgetControl(options=['positio…

In [ ]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['green', 'blue', 'red', 'yellow', 'magenta','purple']

# Reclassify the map
#result = result.remap([0, 1, 2, 3, 4,5], [1, 2, 3, 4, 5,6])
result=result.remap([0,1,2,3,4,5],[1,2,3,4,5,6])

map4.addLayer(result, {'min': 1, 'max': 6, 'palette': legend_colors}, 'Labelled clusters')
map4.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
map4

The legend colors must be a list of tuples.


Map(bottom=15487.0, center=[13.036669323115246, 102.59033203125001], controls=(WidgetControl(options=['positio…